In [ ]:
import io
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plot
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler


In [ ]:
import pandas as pd

dfa = pd.read_csv(r'...')

In [ ]:
df1 = dfa.drop(columns = ['SubjectID', 'VideoID','predefinedlabel'])
df1.dropna()
df = df1.sample(frac =0.8)
df['user-definedlabeln'].value_counts()

**Feature Preprocessing**


In [ ]:
# Splitting into train, val and test set -- 80-10-10 split

# First, an 70-30 split
train_df, test_df = train_test_split(df1, test_size = 0.3, random_state = 113)

# Then split the 20% into half
val_df, test_df = train_test_split(test_df, test_size = 0.5, random_state = 113)

ic = df.columns.tolist()
ic.remove('user-definedlabeln')

oc = ['user-definedlabeln']

ytrain = train_df[oc]
X_train = train_df.drop(columns = oc)

yval = val_df[oc]
X_val = val_df.drop(columns = oc)

ytest = test_df[oc]
X_test = test_df.drop(columns = oc)


**SVM training using default hyperparameters**

In [ ]:
from sklearn import  svm
svm = svm.SVC()
#params = svm.get_params()
#params_df = pd.DataFrame(params, index=[0])
#params_df.T
svm.fit(X_train,ytrain)
print(f'The accuracy score of the model is {svm.score(X_test,ytest.values.ravel()):.5f}')

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy score of the model is 0.58117


**Decomposing Bias and Variance**

*passing pandas dataframe to numpy array as the mlxtend package does not support dataframe*

In [ ]:
X_train=X_train.values

In [ ]:
X_test=X_test.values

In [ ]:
ytrain=ytrain.values

In [ ]:
ytest=ytest.values

**Decomposing Bias & Variance via SVM**

In [ ]:
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        svm, X_train, ytrain, X_test, ytest,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)

 **ANOVA *f* Test to select Strongly correlated Features**

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
# configure to select all features
fs = SelectKBest(score_func=f_classif, k='all')
# learn relationship from training data
fs.fit(X_train, ytrain)
# transform train input data
X_train_fs = fs.transform(X_train)
# transform test input data
X_test_fs = fs.transform(X_test)

In [ ]:
# what are scores for the features
for i in range(len(fs.scores_)):
 print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
plot.bar([i for i in range(len(fs.scores_))], fs.scores_)
plot.show()